In [1]:
import sys
import numpy as np
import pandas as pd

from keras import Sequential
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional,Flatten
from keras import optimizers
from keras import regularizers
from keras.utils import np_utils
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

DATA_DIR = "./"

C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# load data from files and save it into lists
def load_data(f, data, pos_or_neg, target):
    for line in f.readlines():
        line_str = ""
        line = line[1:-2].split(r", ")
        for i in range(len(line)):
            line[i] = line[i][1:-1]
            line_str += str(line[i]+" ")
        data.append(line_str)
        target.append(pos_or_neg)
    
data = []
target = []

for file_name in ["training_pos.csv",
          "training_neg.csv",
          "validation_pos.csv",
          "validation_neg.csv",
          "test_pos.csv",
          "test_neg.csv"]:
    f = open(DATA_DIR + file_name,'r')
    pos_or_neg = 1 if 'pos' in file_name else 0
    load_data(f,data, pos_or_neg, target)
    f.close()


In [3]:
word_seq = [text_to_word_sequence(sent) for sent in data]
# we should be safe using MAX _SENT _LEN as 90th Percentile Sentence Length
MAX_SENT_LEN = int(np.percentile([len(seq) for seq in word_seq], 90))
print('90th Percentile Sentence Length:', MAX_SENT_LEN)

# cut every sentence  according to MAX_SENT_LEN
data = [' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq]

# Convert the sequence of words to sequnce of indices
MAX_VOCAB_SIZE = 80000
tokenizer = Tokenizer(MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(data)
data = tokenizer.texts_to_sequences(data)
data = pad_sequences(data, maxlen=MAX_SENT_LEN, padding='post', truncating='post')


90th Percentile Sentence Length: 23


In [5]:
# split data into train, validation, test
length = len(data)
train_data = data[:int(length*0.8)]
val_data = data[int(length*0.9):]
test_data = data[int(length*0.8):int(length*0.9)]
train_target = target[:int(length*0.8)]
val_target = target[int(length*0.9):]
test_target = target[int(length*0.8):int(length*0.9)]

train = list(zip(train_data, train_target))
import random
random.shuffle(train)
train_data[:], train_target[:] = zip(*train)


In [6]:
train_target = np_utils.to_categorical(train_target, 2)
test_target =  np_utils.to_categorical(test_target, 2)
val_target = np_utils.to_categorical(val_target, 2)

In [7]:
W2V_DIR = 'W2V_DIR'
embeddings = Word2Vec.load(DATA_DIR + W2V_DIR)
print('Dimension of w2v:', embeddings.vector_size)
EMBEDDING_DIM = embeddings.vector_size

C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Dimension of w2v: 300


In [8]:
# Create an embedding matrix containing only the word's in our vocabulary
# If the word does not have a pre-trained embedding, then randomly initialize the embedding
embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1, EMBEDDING_DIM)) # +1 is because the matrix indices start with 0

for word, i in tokenizer.word_index.items(): # i=0 is the embedding for the zero padding
    try:
        embeddings_vector = embeddings[word]
    except KeyError:
        embeddings_vector = None
    if embeddings_vector is not None:
        embeddings_matrix[i] = embeddings_vector
        
del embeddings


C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [9]:
def sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate):
    
    # Build a sequential model by stacking neural net units 
    model = Sequential()
    # Input layer of embeddings
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1,
                              output_dim=EMBEDDING_DIM,
                              weights = [embeddings_matrix], trainable=True, name='word_embedding_layer', 
                              mask_zero=False,input_length=MAX_SENT_LEN))
#     model.add(LSTM(128, return_sequences=False, name='lstm_layer'))
    model.add(Flatten())
    
    if(dropoutFlag):
        model.add(Dropout(dropoutRate))
        
    if(l2normFlag):
        model.add(Dense(units = 30,activation = hidden_actfunc, name = 'hidden_layer',kernel_regularizer=regularizers.l2(l=0.01)))
    else:
        model.add(Dense(units = 30,activation = hidden_actfunc, name = 'hidden_layer'))
    
    if(dropoutFlag):
        model.add(Dropout(dropoutRate))
    
    if(l2normFlag):
        model.add(Dense(2, activation='softmax', name='output_layer',kernel_regularizer=regularizers.l2(l=0.01)))
    else:
        model.add(Dense(2, activation='softmax', name='output_layer'))

    # Use cross-entropy as the loss function
    model.compile(loss='categorical_crossentropy',optimizer ='adam',  metrics=['accuracy'])
    return model
    


In [10]:
# from keras.utils import plot_model
# from IPython.display import Image
# plot_model(model, to_file='basic_lstm_classifier.png', show_layer_names=True, show_shapes=True)
# Image('basic_lstm_classifier.png')

In [11]:
# # train data
# BATCH_SIZE = 128
# N_EPOCHS = 10
# dropoutRate = 0.5
# for hidden_actfunc in ["relu", "sigmoid", "tanh"]:
#     for l2normFlag in [True, False]:
#         for dropoutFlag in [True, False]:
#             print("----------------------------------------------------------------------------------")
#             print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
#                       "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
#             for dropoutRate in np.arange(0.4, 1 , 0.2):
#                 model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
#                 model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
                
                

In [12]:
BATCH_SIZE = 128
N_EPOCHS = 10

## relu

In [13]:
# hidden_layer = relu, l2norm = True, dropout = True
hidden_actfunc = "relu"
l2normFlag = True
dropoutFlag = True
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))

W0625 12:20:14.654095 113336 deprecation_wrapper.py:119] From C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0625 12:20:14.682020 113336 deprecation_wrapper.py:119] From C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0625 12:20:14.691023 113336 deprecation_wrapper.py:119] From C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0625 12:20:14.701988 113336 deprecation_wrapper.py:119] From C:\Users\h5weng\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please

----------------------------------------------------------------------------------
hidden_actfunc: relu   l2normFlag: True   dropoutFlag: True   dropoutRate 0.5
Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 131s 204us/step - loss: 0.7795 - acc: 0.5416 - val_loss: 0.7189 - val_acc: 0.6640
Epoch 2/10
640000/640000 [==============================] - 129s 202us/step - loss: 0.7454 - acc: 0.5826 - val_loss: 0.6916 - val_acc: 0.7140
Epoch 3/10
640000/640000 [==============================] - 129s 202us/step - loss: 0.7396 - acc: 0.5992 - val_loss: 0.6745 - val_acc: 0.7333
Epoch 4/10
640000/640000 [==============================] - 130s 203us/step - loss: 0.7342 - acc: 0.6081 - val_loss: 0.6603 - val_acc: 0.7564
Epoch 5/10
640000/640000 [==============================] - 129s 202us/step - loss: 0.7301 - acc: 0.6130 - val_loss: 0.6594 - val_acc: 0.7639
Epoch 6/10
640000/640000 [==============================] - 130s 203us/step - 

In [14]:
# hidden_layer = relu, l2norm = True, dropout = False
hidden_actfunc = "relu"
l2normFlag = True
dropoutFlag = False
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))       

----------------------------------------------------------------------------------
hidden_actfunc: relu   l2normFlag: True   dropoutFlag: False   dropoutRate 0.5
Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 128s 201us/step - loss: 0.7305 - acc: 0.5818 - val_loss: 0.6240 - val_acc: 0.7260
Epoch 2/10
640000/640000 [==============================] - 127s 199us/step - loss: 0.6782 - acc: 0.6204 - val_loss: 0.5905 - val_acc: 0.7615
Epoch 3/10
640000/640000 [==============================] - 127s 199us/step - loss: 0.6706 - acc: 0.6312 - val_loss: 0.5895 - val_acc: 0.7711
Epoch 4/10
640000/640000 [==============================] - 127s 199us/step - loss: 0.6658 - acc: 0.6364 - val_loss: 0.5726 - val_acc: 0.7792
Epoch 5/10
640000/640000 [==============================] - 127s 199us/step - loss: 0.6625 - acc: 0.6400 - val_loss: 0.5650 - val_acc: 0.7833
Epoch 6/10
640000/640000 [==============================] - 128s 199us/step -

In [15]:
# hidden_layer = relu, l2norm = False, dropout = True
hidden_actfunc = "relu"
l2normFlag = False
dropoutFlag = True
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))       

----------------------------------------------------------------------------------
hidden_actfunc: relu   l2normFlag: False   dropoutFlag: True   dropoutRate 0.5
Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 123s 192us/step - loss: 0.6717 - acc: 0.5830 - val_loss: 0.5816 - val_acc: 0.7564
Epoch 2/10
640000/640000 [==============================] - 122s 191us/step - loss: 0.6549 - acc: 0.6155 - val_loss: 0.5626 - val_acc: 0.7722
Epoch 3/10
640000/640000 [==============================] - 122s 191us/step - loss: 0.6493 - acc: 0.6255 - val_loss: 0.5825 - val_acc: 0.7639
Epoch 4/10
640000/640000 [==============================] - 122s 191us/step - loss: 0.6452 - acc: 0.6316 - val_loss: 0.5581 - val_acc: 0.7819
Epoch 5/10
640000/640000 [==============================] - 122s 191us/step - loss: 0.6415 - acc: 0.6361 - val_loss: 0.5496 - val_acc: 0.7843
Epoch 6/10
640000/640000 [==============================] - 122s 191us/step -

In [16]:
# hidden_layer = relu, l2norm = False, dropout = False
hidden_actfunc = "relu"
l2normFlag = False
dropoutFlag = False
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))            

----------------------------------------------------------------------------------
hidden_actfunc: relu   l2normFlag: False   dropoutFlag: False   dropoutRate 0.5
Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 120s 188us/step - loss: 0.6511 - acc: 0.6163 - val_loss: 0.5342 - val_acc: 0.7834
Epoch 2/10
640000/640000 [==============================] - 119s 187us/step - loss: 0.6301 - acc: 0.6472 - val_loss: 0.5267 - val_acc: 0.7905
Epoch 3/10
640000/640000 [==============================] - 119s 187us/step - loss: 0.6175 - acc: 0.6607 - val_loss: 0.5142 - val_acc: 0.7883
Epoch 4/10
640000/640000 [==============================] - 119s 187us/step - loss: 0.6041 - acc: 0.6725 - val_loss: 0.5181 - val_acc: 0.7800
Epoch 5/10
640000/640000 [==============================] - 119s 187us/step - loss: 0.5904 - acc: 0.6832 - val_loss: 0.5167 - val_acc: 0.7732
Epoch 6/10
640000/640000 [==============================] - 119s 187us/step 

## sigmoid

In [17]:
# hidden_layer = sigmoid, l2norm = True, dropout = True
hidden_actfunc = "sigmoid" 
l2normFlag = True
dropoutFlag = True
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))

----------------------------------------------------------------------------------
hidden_actfunc: sigmoid   l2normFlag: True   dropoutFlag: True   dropoutRate 0.5
Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 123s 192us/step - loss: 0.7359 - acc: 0.5033 - val_loss: 0.6999 - val_acc: 0.5000
Epoch 2/10
640000/640000 [==============================] - 122s 190us/step - loss: 0.7000 - acc: 0.5011 - val_loss: 0.7003 - val_acc: 0.5000
Epoch 3/10
640000/640000 [==============================] - 122s 190us/step - loss: 0.7003 - acc: 0.5030 - val_loss: 0.7016 - val_acc: 0.5145
Epoch 4/10
640000/640000 [==============================] - 122s 190us/step - loss: 0.7029 - acc: 0.5089 - val_loss: 0.7070 - val_acc: 0.5284
Epoch 5/10
640000/640000 [==============================] - 122s 190us/step - loss: 0.7200 - acc: 0.5501 - val_loss: 0.7070 - val_acc: 0.6833
Epoch 6/10
640000/640000 [==============================] - 122s 190us/step

In [18]:
# hidden_layer = sigmoid, l2norm = True, dropout = False
hidden_actfunc = "sigmoid"
l2normFlag = True
dropoutFlag = False
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))

----------------------------------------------------------------------------------
hidden_actfunc: sigmoid   l2normFlag: True   dropoutFlag: False   dropoutRate 0.5Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 121s 188us/step - loss: 0.7194 - acc: 0.5029 - val_loss: 0.6935 - val_acc: 0.5000
Epoch 2/10
640000/640000 [==============================] - 120s 187us/step - loss: 0.6940 - acc: 0.5008 - val_loss: 0.6941 - val_acc: 0.5000
Epoch 3/10
640000/640000 [==============================] - 120s 187us/step - loss: 0.6939 - acc: 0.5007 - val_loss: 0.6945 - val_acc: 0.5397
Epoch 4/10
640000/640000 [==============================] - 120s 187us/step - loss: 0.6937 - acc: 0.5002 - val_loss: 0.6934 - val_acc: 0.6782
Epoch 5/10
640000/640000 [==============================] - 120s 187us/step - loss: 0.6935 - acc: 0.5401 - val_loss: 0.6933 - val_acc: 0.7354
Epoch 6/10
640000/640000 [==============================] - 120s 187us/step

In [19]:
# hidden_layer = sigmoid, l2norm = False, dropout = True
hidden_actfunc = "sigmoid"
l2normFlag = False
dropoutFlag = True
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))

----------------------------------------------------------------------------------
hidden_actfunc: sigmoid   l2normFlag: False   dropoutFlag: True   dropoutRate 0.5
Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 123s 192us/step - loss: 0.6655 - acc: 0.5938 - val_loss: 0.5595 - val_acc: 0.7690
Epoch 2/10
640000/640000 [==============================] - 122s 190us/step - loss: 0.6477 - acc: 0.6240 - val_loss: 0.5368 - val_acc: 0.7779
Epoch 3/10
640000/640000 [==============================] - 122s 190us/step - loss: 0.6418 - acc: 0.6330 - val_loss: 0.5381 - val_acc: 0.7887
Epoch 4/10
640000/640000 [==============================] - 122s 190us/step - loss: 0.6371 - acc: 0.6405 - val_loss: 0.5258 - val_acc: 0.7901
Epoch 5/10
640000/640000 [==============================] - 122s 190us/step - loss: 0.6336 - acc: 0.6443 - val_loss: 0.5293 - val_acc: 0.7935
Epoch 6/10
640000/640000 [==============================] - 122s 190us/ste

In [20]:
# hidden_layer = sigmoid, l2norm = False, dropout = False
hidden_actfunc = "sigmoid"
l2normFlag = False
dropoutFlag = False
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))

----------------------------------------------------------------------------------
hidden_actfunc: sigmoid   l2normFlag: False   dropoutFlag: False   dropoutRate 0.5
Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 120s 188us/step - loss: 0.6509 - acc: 0.6166 - val_loss: 0.5399 - val_acc: 0.7765
Epoch 2/10
640000/640000 [==============================] - 119s 186us/step - loss: 0.6301 - acc: 0.6468 - val_loss: 0.5184 - val_acc: 0.7841
Epoch 3/10
640000/640000 [==============================] - 119s 186us/step - loss: 0.6192 - acc: 0.6591 - val_loss: 0.5174 - val_acc: 0.7887
Epoch 4/10
640000/640000 [==============================] - 119s 186us/step - loss: 0.6091 - acc: 0.6690 - val_loss: 0.5077 - val_acc: 0.7855
Epoch 5/10
640000/640000 [==============================] - 119s 186us/step - loss: 0.5989 - acc: 0.6780 - val_loss: 0.5158 - val_acc: 0.7792
Epoch 6/10
640000/640000 [==============================] - 119s 186us/st

## tanh

In [21]:
# hidden_layer = tanh, l2norm = True, dropout = True
hidden_actfunc = "sigmoid"
l2normFlag = True
dropoutFlag = True
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))

----------------------------------------------------------------------------------
hidden_actfunc: sigmoid   l2normFlag: True   dropoutFlag: True   dropoutRate 0.5
Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 122s 191us/step - loss: 0.7307 - acc: 0.5031 - val_loss: 0.6988 - val_acc: 0.5000
Epoch 2/10
640000/640000 [==============================] - 121s 189us/step - loss: 0.6999 - acc: 0.5010 - val_loss: 0.7013 - val_acc: 0.5000
Epoch 3/10
640000/640000 [==============================] - 123s 192us/step - loss: 0.7000 - acc: 0.5019 - val_loss: 0.7002 - val_acc: 0.5117
Epoch 4/10
640000/640000 [==============================] - 125s 195us/step - loss: 0.7027 - acc: 0.5085 - val_loss: 0.6984 - val_acc: 0.6201
Epoch 5/10
640000/640000 [==============================] - 124s 195us/step - loss: 0.7191 - acc: 0.5481 - val_loss: 0.7060 - val_acc: 0.6666
Epoch 6/10
640000/640000 [==============================] - 128s 200us/step

In [22]:
# hidden_layer = tanh, l2norm = True, dropout = False
hidden_actfunc = "tanh"
l2normFlag = True
dropoutFlag = False
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))

----------------------------------------------------------------------------------
hidden_actfunc: tanh   l2normFlag: True   dropoutFlag: False   dropoutRate 0.5
Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 128s 200us/step - loss: 0.7583 - acc: 0.5656 - val_loss: 0.6652 - val_acc: 0.7044
Epoch 2/10
640000/640000 [==============================] - 128s 199us/step - loss: 0.7078 - acc: 0.6079 - val_loss: 0.6407 - val_acc: 0.7316
Epoch 3/10
640000/640000 [==============================] - 131s 205us/step - loss: 0.7031 - acc: 0.6206 - val_loss: 0.6365 - val_acc: 0.7447
Epoch 4/10
640000/640000 [==============================] - 129s 202us/step - loss: 0.6979 - acc: 0.6281 - val_loss: 0.6186 - val_acc: 0.7637
Epoch 5/10
640000/640000 [==============================] - 129s 202us/step - loss: 0.6940 - acc: 0.6330 - val_loss: 0.5977 - val_acc: 0.7697
Epoch 6/10
640000/640000 [==============================] - 130s 202us/step -

In [27]:
# hidden_layer = tanh, l2norm = False, dropout = True
hidden_actfunc = "tanh"
l2normFlag = False
dropoutFlag = True
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))

----------------------------------------------------------------------------------
hidden_actfunc: tanh   l2normFlag: False   dropoutFlag: True   dropoutRate 0.5
Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 130s 203us/step - loss: 0.6764 - acc: 0.5847 - val_loss: 0.5582 - val_acc: 0.7603
Epoch 2/10
640000/640000 [==============================] - 131s 205us/step - loss: 0.6514 - acc: 0.6180 - val_loss: 0.5410 - val_acc: 0.7780
Epoch 3/10
640000/640000 [==============================] - 131s 205us/step - loss: 0.6444 - acc: 0.6293 - val_loss: 0.5231 - val_acc: 0.7845
Epoch 4/10
640000/640000 [==============================] - 131s 205us/step - loss: 0.6406 - acc: 0.6353 - val_loss: 0.5256 - val_acc: 0.7904
Epoch 5/10
640000/640000 [==============================] - 131s 205us/step - loss: 0.6376 - acc: 0.6396 - val_loss: 0.5248 - val_acc: 0.7915
Epoch 6/10
640000/640000 [==============================] - 131s 205us/step -

In [28]:
# hidden_layer = tanh, l2norm = False, dropout = False
hidden_actfunc = "tanh"
l2normFlag = False
dropoutFlag = False
dropoutRate = 0.5
model = sequential_model (hidden_actfunc, l2normFlag, dropoutFlag, dropoutRate)
print("----------------------------------------------------------------------------------")
print("hidden_actfunc: "+hidden_actfunc, "  l2normFlag: " + str(l2normFlag), 
      "  dropoutFlag: "+str(dropoutFlag), "  dropoutRate " + str(dropoutRate))
model.fit(train_data, train_target,batch_size=BATCH_SIZE,epochs=N_EPOCHS, validation_data =(val_data, val_target))
scores = model.evaluate(test_data, test_target)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))

----------------------------------------------------------------------------------
hidden_actfunc: tanh   l2normFlag: False   dropoutFlag: False   dropoutRate 0.5
Train on 640000 samples, validate on 80000 samples
Epoch 1/10
640000/640000 [==============================] - 124s 194us/step - loss: 0.6568 - acc: 0.6089 - val_loss: 0.5386 - val_acc: 0.7688
Epoch 2/10
640000/640000 [==============================] - 123s 192us/step - loss: 0.6357 - acc: 0.6392 - val_loss: 0.5306 - val_acc: 0.7786
Epoch 3/10
640000/640000 [==============================] - 122s 191us/step - loss: 0.6270 - acc: 0.6500 - val_loss: 0.5222 - val_acc: 0.7829
Epoch 4/10
640000/640000 [==============================] - 123s 193us/step - loss: 0.6206 - acc: 0.6579 - val_loss: 0.5210 - val_acc: 0.7844
Epoch 5/10
640000/640000 [==============================] - 134s 210us/step - loss: 0.6146 - acc: 0.6636 - val_loss: 0.5155 - val_acc: 0.7808
Epoch 6/10
640000/640000 [==============================] - 135s 211us/step 